Summary the adjs for Overall Service for each business.

In [1]:
import numpy as np
import pandas as pd
import ast
from textblob import TextBlob
from nltk import pos_tag

wait/waiting(0,1)
waiter/waitress/staff 2,3,7, serve/service/services/server/servers 4,5,6,,40,41
paid/price/prices/pay/money/bill 8,9,10,33,38,39

In [2]:
words = ["wait", "waiting", 
         "waiter", "waitress", "staff", "serve", "service", "services", "server", "servers",
         "paid", "price","prices","pay", "money", "bill"]

In [3]:
#24 2-grams phrases
phrases_2 = [('dim','sum'), ('fried','rice'), ('orange', 'chicken'), ('chow', 'mein'), ('pad', 'thai'), ('sour', 'soup'), ('hot', 'pot'), ('spring', 'rolls'), ('lo', 'mein'), ('bbq', 'pork'), ('mongolian', 'beef'), ('egg', 'roll'), ('wonton', 'soup'), ('soy', 'sauce'), ('ice', 'cream'), ('pork', 'belly'), ('milk', 'tea'), ('fried', 'chicken'),  ('sour', 'chicken'), ('white', 'rice'), ('sesame', 'chicken'), ('chicken', 'wings'), ('peking', 'duck'), ('bubble', 'tea')]

In [4]:
#10 -grams phrases
phrases_3 = [('egg', 'drop', 'soup'), ('beef', 'noodle', 'soup'), ('kung', 'pao', 'chicken'), ('chicken', 'fried', 'rice'), ('pork', 'fried', 'rice'), ('black', 'bean', 'sauce'), ('shrimp', 'fried', 'rice'), ('xiao', 'long', 'bao'), ('egg', 'foo', 'young'), ('dan', 'dan', 'noodles')]

In [5]:
file = pd.read_csv("Chinese_review.csv")
reviews_token = file["text_token"].tolist()
reviews_token = [ast.literal_eval(review) for review in reviews_token]

bigram = file["bigram"]
bigram_token = bigram.tolist()
bigram_token = [ast.literal_eval(review) for review in bigram_token]

trigram = file["trigram"]
trigram_token = trigram.tolist()
trigram_token = [ast.literal_eval(review) for review in trigram_token]

In [6]:
data_phrases_1 = []

for i in reviews_token:
    score = []
    for phrase in words:
        if phrase in i:
            score.append(1)
        else:
            score.append(0)
    
    data_phrases_1.append(score)
    
#data of phrases(2 words)
data_phrases_2 = []

for i in bigram_token:
    score = []
    for phrase in phrases_2:
        if phrase in i:
            score.append(1)
        else:
            score.append(0)
    
    data_phrases_2.append(score)
    
#data of phrases(3 words)
data_phrases_3 = []

for i in trigram_token:
    score = []
    for phrase in phrases_3:
        if phrase in i:
            score.append(1)
        else:
            score.append(0)
    
    data_phrases_3.append(score)

In [7]:
X1 = np.array(data_phrases_1)
X2 = np.array(data_phrases_2)
X3 = np.array(data_phrases_3)

In [9]:
index1 = [0,1]
index2 = [2,3,4,5,6,7,8,9]
index3 = [10,11,12,13,14,15]

In [14]:
total_number = len(reviews_token)

In [16]:
X1_new = []

for i in range(total_number):
    x = list()
    if any(X1[i,index1]):
        x.append(1)
    else:
        x.append(0)
    
    if any(X1[i,index2]):
        x.append(1)
    else:
        x.append(0)
        
    if any(X1[i,index3]):
        x.append(1)
    else:
        x.append(0)
        
    X1_new.append(x)

In [17]:
X1_new = np.array(X1_new)

In [20]:
## The X matrix for Overall Service + two-word food + three-word food

X = np.hstack((X1_new, X2, X3))

In [21]:
colnames = ["wait", "service", "price"] + phrases_2 + phrases_3
X_df = pd.DataFrame(X, columns=colnames)

In [22]:
X_df.to_csv("X_df.csv")

In [23]:
waitIndex = [0,1] ## wait, waiting
serviceIndex = [2,3,4,5,6,7,40,41] ## waiter, waitress, staff, serve, service, services, server, servers;
payIndex = [8,9,10,33,38,39] ## paid, price,prices,pay, money, bill

In [24]:
ratings = file["stars"]

In [26]:
binary_ratings = [0] * total_number

for i in range(total_number):
    if ratings[i] > 3.0:
        binary_ratings[i] = 1

In [41]:
total_adjective_1_df = pd.read_csv("total_adjective_1_df.csv")

In [42]:
total_adjective_1_df  = total_adjective_1_df.iloc[:,1:]

In [65]:
bus1_neg = bus1[bus1["binary_ratings"]==0]
bus1_pos = bus1[bus1["binary_ratings"]==1]

In [169]:
### For each business, combine the adj for positive reviews together, combine the adj for negative reviews together.

waitNegTotal = list()
serviceNegTotal = list()
payNegTotal = list()

waitPosTotal = list()
servicePosTotal = list()
payPosTotal = list()

numberOfReview = list()
numberOfNegReview = list()
numberOfPosReview = list()


for index in range(4434):  ## number of Business

    print("business_index:", index)
    indexRange = np.where(total_adjective_1_df["business_index"] == index)[0]
    temp1 = total_adjective_1_df.iloc[indexRange,waitIndex]
    temp2 = total_adjective_1_df.iloc[indexRange,serviceIndex]
    temp3 = total_adjective_1_df.iloc[indexRange,payIndex]
    ratings = [binary_ratings[i] for i in indexRange]
    
    neg_index = np.where(np.array(ratings) == 0)[0]
    pos_index = np.where(np.array(ratings) == 1)[0]
    
    numberOfReview.append(len(indexRange))
    numberOfNegReview.append(len(neg_index))
    numberOfPosReview.append(len(pos_index)
                            )
   ### all adjs for negative reviews
    n1 = list()
    n2 = list()
    n3 = list()
    
    
    for i in neg_index:
        ## print(i)
        for j in range(len(waitIndex)):
            if type(temp1.iloc[i,j]) != str:
                continue
            
            item = ast.literal_eval(temp1.iloc[i,j])

            if item != None and item != []:
                if len(item) > 1:
                    word = item[0:-1]

                    for w in word:
                        n1.append(w)
        
        for j in range(len(serviceIndex)):      
            if type(temp2.iloc[i,j]) != str:
                continue            
            item = ast.literal_eval(temp2.iloc[i,j])
            if item != None and item != []:
                if len(item) > 1:
                    word = item[0:-1]

                    for w in word:
                        n2.append(w)
                        
        for j in range(len(payIndex)):      
            if type(temp3.iloc[i,j]) != str:
                continue            
            item = ast.literal_eval(temp3.iloc[i,j])
            if item != None and item != []:
                if len(item) > 1:
                    word = item[0:-1]

                    for w in word:
                        n3.append(w)
    
      
    
    waitNegTotal.append(n1)
    serviceNegTotal.append(n2)
    payNegTotal.append(n3)
    
### all adjs for positive reviews
    p1 = list()
    p2 = list()
    p3 = list()
    
   
    for i in pos_index:
        for j in range(len(waitIndex)):
            if type(temp1.iloc[i,j]) != str:
                continue
            
            item = ast.literal_eval(temp1.iloc[i,j])

            if item != None and item != []:
                if len(item) > 1:
                    word = item[0:-1]

                    for w in word:
                        p1.append(w)
        
        for j in range(len(serviceIndex)):      
            if type(temp2.iloc[i,j]) != str:
                continue            
            item = ast.literal_eval(temp2.iloc[i,j])
            if item != None and item != []:
                if len(item) > 1:
                    word = item[0:-1]

                    for w in word:
                        p2.append(w)
                        
        for j in range(len(payIndex)):      
            if type(temp3.iloc[i,j]) != str:
                continue            
            item = ast.literal_eval(temp3.iloc[i,j])
            if item != None and item != []:
                if len(item) > 1:
                    word = item[0:-1]

                    for w in word:
                        p3.append(w)
                
    waitPosTotal.append(p1)
    servicePosTotal.append(p2)
    payPosTotal.append(p3)
 



business_index: 0
business_index: 1
business_index: 2
business_index: 3
business_index: 4
business_index: 5
business_index: 6
business_index: 7
business_index: 8
business_index: 9
business_index: 10
business_index: 11
business_index: 12
business_index: 13
business_index: 14
business_index: 15
business_index: 16
business_index: 17
business_index: 18
business_index: 19
business_index: 20
business_index: 21
business_index: 22
business_index: 23
business_index: 24
business_index: 25
business_index: 26
business_index: 27
business_index: 28
business_index: 29
business_index: 30
business_index: 31
business_index: 32
business_index: 33
business_index: 34
business_index: 35
business_index: 36
business_index: 37
business_index: 38
business_index: 39
business_index: 40
business_index: 41
business_index: 42
business_index: 43
business_index: 44
business_index: 45
business_index: 46
business_index: 47
business_index: 48
business_index: 49
business_index: 50
business_index: 51
business_index: 52
bus

business_index: 416
business_index: 417
business_index: 418
business_index: 419
business_index: 420
business_index: 421
business_index: 422
business_index: 423
business_index: 424
business_index: 425
business_index: 426
business_index: 427
business_index: 428
business_index: 429
business_index: 430
business_index: 431
business_index: 432
business_index: 433
business_index: 434
business_index: 435
business_index: 436
business_index: 437
business_index: 438
business_index: 439
business_index: 440
business_index: 441
business_index: 442
business_index: 443
business_index: 444
business_index: 445
business_index: 446
business_index: 447
business_index: 448
business_index: 449
business_index: 450
business_index: 451
business_index: 452
business_index: 453
business_index: 454
business_index: 455
business_index: 456
business_index: 457
business_index: 458
business_index: 459
business_index: 460
business_index: 461
business_index: 462
business_index: 463
business_index: 464
business_index: 465


business_index: 832
business_index: 833
business_index: 834
business_index: 835
business_index: 836
business_index: 837
business_index: 838
business_index: 839
business_index: 840
business_index: 841
business_index: 842
business_index: 843
business_index: 844
business_index: 845
business_index: 846
business_index: 847
business_index: 848
business_index: 849
business_index: 850
business_index: 851
business_index: 852
business_index: 853
business_index: 854
business_index: 855
business_index: 856
business_index: 857
business_index: 858
business_index: 859
business_index: 860
business_index: 861
business_index: 862
business_index: 863
business_index: 864
business_index: 865
business_index: 866
business_index: 867
business_index: 868
business_index: 869
business_index: 870
business_index: 871
business_index: 872
business_index: 873
business_index: 874
business_index: 875
business_index: 876
business_index: 877
business_index: 878
business_index: 879
business_index: 880
business_index: 881


business_index: 1243
business_index: 1244
business_index: 1245
business_index: 1246
business_index: 1247
business_index: 1248
business_index: 1249
business_index: 1250
business_index: 1251
business_index: 1252
business_index: 1253
business_index: 1254
business_index: 1255
business_index: 1256
business_index: 1257
business_index: 1258
business_index: 1259
business_index: 1260
business_index: 1261
business_index: 1262
business_index: 1263
business_index: 1264
business_index: 1265
business_index: 1266
business_index: 1267
business_index: 1268
business_index: 1269
business_index: 1270
business_index: 1271
business_index: 1272
business_index: 1273
business_index: 1274
business_index: 1275
business_index: 1276
business_index: 1277
business_index: 1278
business_index: 1279
business_index: 1280
business_index: 1281
business_index: 1282
business_index: 1283
business_index: 1284
business_index: 1285
business_index: 1286
business_index: 1287
business_index: 1288
business_index: 1289
business_inde

business_index: 1647
business_index: 1648
business_index: 1649
business_index: 1650
business_index: 1651
business_index: 1652
business_index: 1653
business_index: 1654
business_index: 1655
business_index: 1656
business_index: 1657
business_index: 1658
business_index: 1659
business_index: 1660
business_index: 1661
business_index: 1662
business_index: 1663
business_index: 1664
business_index: 1665
business_index: 1666
business_index: 1667
business_index: 1668
business_index: 1669
business_index: 1670
business_index: 1671
business_index: 1672
business_index: 1673
business_index: 1674
business_index: 1675
business_index: 1676
business_index: 1677
business_index: 1678
business_index: 1679
business_index: 1680
business_index: 1681
business_index: 1682
business_index: 1683
business_index: 1684
business_index: 1685
business_index: 1686
business_index: 1687
business_index: 1688
business_index: 1689
business_index: 1690
business_index: 1691
business_index: 1692
business_index: 1693
business_inde

...

business_index: 2437
business_index: 2438
business_index: 2439
business_index: 2440
business_index: 2441
business_index: 2442
business_index: 2443
business_index: 2444
business_index: 2445
business_index: 2446
business_index: 2447
business_index: 2448
business_index: 2449
business_index: 2450
business_index: 2451
business_index: 2452
business_index: 2453
business_index: 2454
business_index: 2455
business_index: 2456
business_index: 2457
business_index: 2458
business_index: 2459
business_index: 2460
business_index: 2461
business_index: 2462
business_index: 2463
business_index: 2464
business_index: 2465
business_index: 2466
business_index: 2467
business_index: 2468
business_index: 2469
business_index: 2470
business_index: 2471
business_index: 2472
business_index: 2473
business_index: 2474
business_index: 2475
business_index: 2476
business_index: 2477
business_index: 2478
business_index: 2479
business_index: 2480
business_index: 2481
business_index: 2482
business_index: 2483
business_inde

business_index: 2835
business_index: 2836
business_index: 2837
business_index: 2838
business_index: 2839
business_index: 2840
business_index: 2841
business_index: 2842
business_index: 2843
business_index: 2844
business_index: 2845
business_index: 2846
business_index: 2847
business_index: 2848
business_index: 2849
business_index: 2850
business_index: 2851
business_index: 2852
business_index: 2853
business_index: 2854
business_index: 2855
business_index: 2856
business_index: 2857
business_index: 2858
business_index: 2859
business_index: 2860
business_index: 2861
business_index: 2862
business_index: 2863
business_index: 2864
business_index: 2865
business_index: 2866
business_index: 2867
business_index: 2868
business_index: 2869
business_index: 2870
business_index: 2871
business_index: 2872
business_index: 2873
business_index: 2874
business_index: 2875
business_index: 2876
business_index: 2877
business_index: 2878
business_index: 2879
business_index: 2880
business_index: 2881
business_inde

business_index: 3236
business_index: 3237
business_index: 3238
business_index: 3239
business_index: 3240
business_index: 3241
business_index: 3242
business_index: 3243
business_index: 3244
business_index: 3245
business_index: 3246
business_index: 3247
business_index: 3248
business_index: 3249
business_index: 3250
business_index: 3251
business_index: 3252
business_index: 3253
business_index: 3254
business_index: 3255
business_index: 3256
business_index: 3257
business_index: 3258
business_index: 3259
business_index: 3260
business_index: 3261
business_index: 3262
business_index: 3263
business_index: 3264
business_index: 3265
business_index: 3266
business_index: 3267
business_index: 3268
business_index: 3269
business_index: 3270
business_index: 3271
business_index: 3272
business_index: 3273
business_index: 3274
business_index: 3275
business_index: 3276
business_index: 3277
business_index: 3278
business_index: 3279
business_index: 3280
business_index: 3281
business_index: 3282
business_inde

business_index: 3635
business_index: 3636
business_index: 3637
business_index: 3638
business_index: 3639
business_index: 3640
business_index: 3641
business_index: 3642
business_index: 3643
business_index: 3644
business_index: 3645
business_index: 3646
business_index: 3647
business_index: 3648
business_index: 3649
business_index: 3650
business_index: 3651
business_index: 3652
business_index: 3653
business_index: 3654
business_index: 3655
business_index: 3656
business_index: 3657
business_index: 3658
business_index: 3659
business_index: 3660
business_index: 3661
business_index: 3662
business_index: 3663
business_index: 3664
business_index: 3665
business_index: 3666
business_index: 3667
business_index: 3668
business_index: 3669
business_index: 3670
business_index: 3671
business_index: 3672
business_index: 3673
business_index: 3674
business_index: 3675
business_index: 3676
business_index: 3677
business_index: 3678
business_index: 3679
business_index: 3680
business_index: 3681
business_inde

business_index: 4036
business_index: 4037
business_index: 4038
business_index: 4039
business_index: 4040
business_index: 4041
business_index: 4042
business_index: 4043
business_index: 4044
business_index: 4045
business_index: 4046
business_index: 4047
business_index: 4048
business_index: 4049
business_index: 4050
business_index: 4051
business_index: 4052
business_index: 4053
business_index: 4054
business_index: 4055
business_index: 4056
business_index: 4057
business_index: 4058
business_index: 4059
business_index: 4060
business_index: 4061
business_index: 4062
business_index: 4063
business_index: 4064
business_index: 4065
business_index: 4066
business_index: 4067
business_index: 4068
business_index: 4069
business_index: 4070
business_index: 4071
business_index: 4072
business_index: 4073
business_index: 4074
business_index: 4075
business_index: 4076
business_index: 4077
business_index: 4078
business_index: 4079
business_index: 4080
business_index: 4081
business_index: 4082
business_inde

In [170]:
summary_df = pd.DataFrame(np.array(numberOfReview), columns=["numberOfReview"])
summary_df["numberOfPosReview"] = numberOfPosReview
summary_df["numberOfNegReview"] = numberOfNegReview

summary_df["waitNegTotal"] =  waitNegTotal
summary_df["waitPosTotal"] =  waitPosTotal

summary_df["serviceNegTotal"] = serviceNegTotal
summary_df["servicePosTotal"] = servicePosTotal

summary_df["payNegTotal"] = payNegTotal
summary_df["payPosTotal"] = payPosTotal


In [209]:
waitTotal = [0] * 4434
serviceTotal = [0] * 4434
payTotal = [0] * 4434

for i in range(4434):
    waitTotal[i] = waitNegTotal[i] + waitPosTotal[i]
    serviceTotal[i] = serviceNegTotal[i] + servicePosTotal[i]
    payTotal[i] = payNegTotal[i] + payPosTotal[i]

In [211]:
summary_df["waitTotal"] = waitTotal
summary_df["serviceTotal"] = serviceTotal
summary_df["payTotal"] = payTotal

Select some adjs then report them.

wait: long/short
waiter: friendly/good/great/nice/attentive/bad/horrible/terrible/pretty/poor/best/well
pay: reasonable/good/great/high/low/affordable/higher/bad/expensive

In [212]:
d0 = Counter(summary_df["waitTotal"][0])

In [215]:
waitWantAdj = ["long", "short"]
serviceWantAdj = ["friendly","good", "great","nice","attentive","bad","horrible","terrible","pretty","poor","best","well"]
payWantAdj = ["reasonable", "good", "great", "high", "low", "affordable", "higher", "bad", "expensive"]

In [236]:
### select the adjs that are common and interpretable for people. Count these adjs for each business. List them in the final result 
### in order to give some intuitive description.

dwList = list()
dsList = list()
dpList = list()

for i in range(4434):
    waitW = waitTotal[i]
    serviceW = serviceTotal[i]
    payW = payTotal[i]
    
    dw = list()
    ds = list()
    dp = list()
    
    for w in waitW:
        if w in waitWantAdj:
            dw.append(w)
            
    for w in serviceW:
        if w in serviceWantAdj:
            ds.append(w)
            
    for w in payW:
        if w in payWantAdj:
            dp.append(w)
            


    dwList.append(Counter(dw).most_common())
    dsList.append(Counter(ds).most_common())
    dpList.append(Counter(dp).most_common())
    
    

In [232]:
Counter(dw).most_common()

[('short', 1)]

In [251]:
business_id = [""] * 4434
ids = file["business_id"]
index = file["business_index"]

In [252]:
for i in range(4434):
    ix = np.where(index == i)[0][0]
    business_id[i] = ids[ix]

In [243]:
DescriptWords = pd.DataFrame(np.array(dwList), columns=["wait"])
DescriptWords["service"] = np.array(dsList)
DescriptWords["price"] = np.array(dpList)

In [254]:
DescriptWords["business_id"] = business_id

In [257]:
DescriptWords["numberOfPosReview"] = numberOfPosReview
DescriptWords["numberOfNegReview"] = numberOfNegReview
DescriptWords["numberOfReview"] = numberOfReview

In [258]:
### Save the file. Use for final analysis.

DescriptWords.to_csv("DescriptWords.csv", index=False)

In [174]:
from collections import Counter

In [240]:
d0 = Counter(summary_df.iloc[0,6])

In [195]:
### No use in final report

# waitNegPro = [0] * 4434
# waitPosPro = [0] * 4434

# serviceNegPro = [0] * 4434
# servicePosPro = [0] * 4434


# payNegPro = [0] * 4434
# payPosPro = [0] * 4434

# for i in range(4434):
    
#     if numberOfNegReview[i] == 0:
#         waitNegPro[i] = -1
#         serviceNegPro[i] = -1
#         payNegPro[i] = -1
#     else:
#         waitNegPro[i] = len(waitNegTotal[i]) / numberOfNegReview[i]
#         serviceNegPro[i] = len(serviceNegTotal[i]) / numberOfNegReview[i]
#         payNegPro[i] = len(payNegTotal[i]) / numberOfNegReview[i]
        
#     if numberOfPosReview[i] == 0:
#         waitPosPro[i] = -1
#         servicePosPro[i] = -1
#         payPosPro[i] == -1
#     else:
#         waitPosPro[i] = len(waitPosTotal[i]) / numberOfPosReview[i]
#         servicePosPro[i] = len(servicePosTotal[i]) / numberOfPosReview[i]
#         payPosPro[i] = len(payPosTotal[i]) / numberOfPosReview[i]
      